In [3]:
import glob
import os
import geopandas as gpd
import pandas as pd
import shutil
img_size = 400

train_val_label = "train2017"

samples_folder='summer_treecanopy'
if 'val' in train_val_label:
    csv = 'test.csv'
elif 'train' in train_val_label:
    csv = 'train_val.csv'
elif 'test' in train_val_label:
    csv = 'final_df.csv'
    samples_folder = 'test'
samples_dir=f'/media/deadman445/disk/PycharmProjects/data/05/{samples_folder}_{img_size}'
df_path = os.path.join(samples_dir, csv)

yolo_base = os.path.join(os.path.dirname(samples_dir),f'{img_size}/coco128')

In [4]:
a = gpd.read_file(df_path)

In [5]:
a

,field_1,Unnamed: 0,Unnamed: 0.1,minx,miny,maxx,maxy,xmin,xmax,ymin,ymax,image_path,label,geometry
0,0,0,0,3328503.5082785,299949.04697027267,3328510.155555656,299959.5328182102,3.3913924964144826,36.62777827819809,207.3359089490259,259.76514863665216,three_trees_afs_summer_9_Baykalsk_CIMA_gpx_H_0...,Tree,None
1,1,1,1,3328503.2658523805,299957.6998585345,3328510.872262953,299967.67438242404,2.179261902347207,40.211314763873816,166.62808787979884,216.50070732750464,three_trees_afs_summer_9_Baykalsk_CIMA_gpx_H_0...,Tree,None
2,2,2,2,3328506.3839339456,299955.6865290579,3328512.1306242207,299961.68657444685,17.769669727422297,46.50312110316008,196.5671277657384,226.5673547104234,three_trees_afs_summer_9_Baykalsk_CIMA_gpx_H_0...,Tree,None
3,3,3,3,3328508.817994219,299957.0489655764,3328515.1111349263,299964.51642204286,29.93997109355405,61.40567463124171,182.41788978572004,219.7551721180207,three_trees_afs_summer_9_Baykalsk_CIMA_gpx_H_0...,Tree,None
4,4,4,4,3328509.162846389,299961.4080282363,3328517.468528173,299969.376616766,31.664231945760548,73.1926408642903,158.11691617011093,197.9598588185036,three_trees_afs_summer_9_Baykalsk_CIMA_gpx_H_0...,Tree,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12480,15116,15116,6506,3328925.3952401862,299552.9437644123,3328931.5827107388,299558.6530978307,21.8262009308678,52.76355369342491,101.73451084643602,130.28117793845013,three_trees_afs_summer_9_Baykalsk_CIMA_gpx_H_4...,Tree,None
12481,15117,15117,6507,3328927.222536858,299555.7932382531,3328932.8785379897,299561.693414538,30.96268429048405,59.2426899480638,86.53292731003603,116.03380873450078,three_trees_afs_summer_9_Baykalsk_CIMA_gpx_H_4...,Tree,None
12482,15118,15118,6508,3328923.465446011,299559.9868544992,3328932.050122681,299567.92387239274,12.177230055443944,55.100613407325,55.38063803629484,95.06572750397028,three_trees_afs_summer_9_Baykalsk_CIMA_gpx_H_4...,Tree,None
12483,15119,15119,6509,3328937.3611611547,299566.43493912136,3328942.7988278195,299571.67062386684,81.65580577310175,108.84413909725843,36.646880665794015,62.82530439319089,three_trees_afs_summer_9_Baykalsk_CIMA_gpx_H_4...,Tree,None


In [6]:
a.groupby(['image_path']).size().max()

403

In [7]:

images_base = os.path.join(yolo_base,'images')
labels_base = os.path.join(yolo_base,'labels')
os.makedirs(yolo_base, exist_ok=True)
os.makedirs(images_base, exist_ok=True)
os.makedirs(labels_base, exist_ok=True)


In [8]:
for name, group in a.groupby(['image_path']):
    # print(group)
    base_name = os.path.splitext(os.path.basename(name))[0]
    group['xmin'] = group['xmin'].astype(float)
    group['xmax'] = group['xmax'].astype(float)
    group['ymin'] = group['ymin'].astype(float)
    group['ymax'] = group['ymax'].astype(float)
    x = (group['xmin'] + group['xmax'])/(2.0*img_size)
    y = (group['ymin'] + group['ymax'])/(2.0*img_size)
    w = (group['xmax'] - group['xmin'])/img_size
    h = (group['ymax'] - group['ymin'])/img_size
    res = pd.concat([x*0,x,y,w,h], axis=1)
    labels_train = os.path.join(labels_base,train_val_label)
    os.makedirs(labels_train, exist_ok=True)
    images_train = os.path.join(images_base,train_val_label)
    os.makedirs(images_train, exist_ok=True)
    res.to_csv(f'{os.path.join(labels_train, base_name)}.txt', header=None, index=None, sep=' ', mode='w+')
    shutil.copy2(os.path.join(samples_dir, name),os.path.join(images_train, name))